In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from dstapi import DstApi


Create objects to interact with API of Denmark Statistics and display a summary table of the datasets informations:

In [6]:
cop_dst = DstApi('FU07')
gdp_dst = DstApi('NRHP')

t_cop = cop_dst.tablesummary(language='en')
t_gdp = gdp_dst.tablesummary(language='en')
display(t_cop,t_gdp)

Table FU07: Consumption by group of consumption, region, price unit and time
Last update: 2023-03-29T08:00:00
Table NRHP: 1-2.1.1 Production, GDP and generation of income by region, transaction, price unit and time
Last update: 2022-10-27T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,KONSUMGRP,47,1000,CONSUMPTION TOTAL,127000,12.7 Other services n.e.c.,False
1,REGION,6,2001,Average Households,6050,Nordjylland,False
2,PRISENHED,2,AARPRIS,Current prices,08PRIS,Real prices,False
3,Tid,28,1994,1994,2021,2021,True


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,18,000,All Denmark,999,Outside regions,False
1,TRANSAKT,9,P1K,P.1 Output,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...,False
2,PRISENHED,4,V_T,"Current prices, (mill. DKK.)",LRG_C,"Pr. capita, 2010-prices, chained values, (1000...",False
3,Tid,29,1993,1993,2021,2021,True


Look up values (rows) that each variable (columns) can take: 

In [10]:
for variable in t_cop['variable name']:
    print(variable+':')
    display(cop_dst.variable_levels(variable, language='en'))

print(f'\n-------------------------------------------------\n')

for variable in t_gdp['variable name']:
    print(variable+':')
    display(gdp_dst.variable_levels(variable, language='en'))

KONSUMGRP:


,id,text
0,1000,CONSUMPTION TOTAL
1,011000,01.1 Food
2,012000,01.2 Non-alcoholic beverages
3,021000,02.1 Alcoholic beverages
4,022000,02.2 Tobacco
5,023000,02.3 Narcotics
6,031000,03.1 Clothing
7,032000,03.2 Footwear
8,041000,04.1 Actual rentals for housing
9,042000,04.2 Imputed rentals for housing


REGION:


,id,text
0,2001,Average Households
1,6010,Hovedstaden
2,6020,Sjælland
3,6030,Syddanmark
4,6040,Midtjylland
5,6050,Nordjylland


PRISENHED:


...

,id,text
0,P1K,P.1 Output
1,P2D,P.2 Intermediate consumption
2,B1GD,B.1g Gross value added
3,D21X31D,D.21-D.31 Taxes less subsidies on products
4,B1GQD,B.1*g Gross domestic product
5,D29X39D,D.29-D.39 Other taxes less subsidies on produc...
6,B1GFD,B.1GF Gross domestic product at factor cost
7,D1D,D.1 Compensation of employees
8,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...


PRISENHED:


,id,text
0,V_T,"Current prices, (mill. DKK.)"
1,V_C,"Pr. capita. Current prices, (1000 DKK.)"
2,LRG_T,"2010-prices, chained values, (mill. DKK.)"
3,LRG_C,"Pr. capita, 2010-prices, chained values, (1000..."


Tid:


,id,text
0,1993,1993
1,1994,1994
2,1995,1995
3,1996,1996
4,1997,1997
5,1998,1998
6,1999,1999
7,2000,2000
8,2001,2001
9,2002,2002


Look up the format of the dictionary of dataset parameters:

In [12]:
par_cop = cop_dst._define_base_params(language='en')
par_gdp = gdp_dst._define_base_params(language='en')

display(par_cop,par_gdp)

{'table': 'fu07',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KONSUMGRP', 'values': ['*']},
  {'code': 'REGION', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

{'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'TRANSAKT', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

Define parameters dictionary to select only specified values (rows) of dataset:

In [15]:
par_cop = {'table': 'fu07',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KONSUMGRP', 'values': ['011000']},
  {'code': 'REGION', 'values': ['2001']},
  {'code': 'PRISENHED', 'values': ['08PRIS']},
  {'code': 'Tid', 'values': ['*']}]}

par_gdp = {'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['000']},
  {'code': 'TRANSAKT', 'values': ['B1GQD']},
  {'code': 'PRISENHED', 'values': ['LRG_T']},
  {'code': 'Tid', 'values': ['*']}]}

# Just took some random parameters: I'll fix it later
# !! Does real price in gdp dataset have same base year as in consumption dataset?

,KONSUMGRP,REGION,PRISENHED,TID,INDHOLD
0,01.1 Food,Average Households,Real prices,2019,33004
1,01.1 Food,Average Households,Real prices,2020,33774
2,01.1 Food,Average Households,Real prices,1994,35028
3,01.1 Food,Average Households,Real prices,1995,35031
4,01.1 Food,Average Households,Real prices,1996,34386


,OMRÅDE,TRANSAKT,PRISENHED,TID,INDHOLD
0,All Denmark,B.1*g Gross domestic product,"2010-prices, chained values, (mill. DKK.)",1993,1332296
1,All Denmark,B.1*g Gross domestic product,"2010-prices, chained values, (mill. DKK.)",1994,1403340
2,All Denmark,B.1*g Gross domestic product,"2010-prices, chained values, (mill. DKK.)",1995,1445828
3,All Denmark,B.1*g Gross domestic product,"2010-prices, chained values, (mill. DKK.)",1996,1487758
4,All Denmark,B.1*g Gross domestic product,"2010-prices, chained values, (mill. DKK.)",1997,1536272


Download datasets using only specified parameters:

In [ ]:
cop = cop_dst.get_data(params=par_cop)
gdp = gdp_dst.get_data(params=par_gdp)

display(cop.head(5),gdp.head(5))

Alternatively we can selectevely download the dataset with only necessary parameters, and then import it. We use this method to import FU07 dataset:

In [ ]:
cop1 = pd.read_csv('FU07.csv')